In [1]:
import pandas as pd

In [2]:
pip install mysql-connector-python

Note: you may need to restart the kernel to use updated packages.


In [3]:
import os

In [4]:
import pandas as pd
import mysql.connector
import os

# List of CSV files and their corresponding table names
csv_files = [
    ('customers.csv', 'customers'),
    ('orders.csv', 'orders'),
    ('sellers.csv', 'sellers'),
    ('products.csv', 'products'),
    ('order_items.csv', 'order_items'),  # Added the missing comma here
    ('payments.csv', 'payments'),
    ('geolocation.csv', 'geolocation')# Added payments.csv for specific handling
]

# Connect to the MySQL database
conn = mysql.connector.connect(
    host='127.0.0.1',
    user='root',
    password='@UmaHustler019',
    database='ecommerce_target'
)
cursor = conn.cursor()

# Folder containing the CSV files
folder_path = 'C:/Users/UMA SHAW/OneDrive/ドキュメント/project intern'

def get_sql_type(dtype):
    if pd.api.types.is_integer_dtype(dtype):
        return 'INT'
    elif pd.api.types.is_float_dtype(dtype):
        return 'FLOAT'
    elif pd.api.types.is_bool_dtype(dtype):
        return 'BOOLEAN'
    elif pd.api.types.is_datetime64_any_dtype(dtype):
        return 'DATETIME'
    else:
        return 'TEXT'

for csv_file, table_name in csv_files:
    file_path = os.path.join(folder_path, csv_file)
    
    # Read the CSV file into a pandas DataFrame
    df = pd.read_csv(file_path)
    
    # Replace NaN with None to handle SQL NULL
    df = df.where(pd.notnull(df), None)
    
      # Debugging: Check for NaN values
    print(f"Processing {csv_file}")
    print(f"NaN values before replacement:\n{df.isnull().sum()}\n")
    
   
    # Clean column names
    df.columns = [col.replace(' ', '_').replace('-', '_').replace('.', '_') for col in df.columns]

    # Generate the CREATE TABLE statement with appropriate data types
    columns = ', '.join([f'`{col}` {get_sql_type(df[col].dtype)}' for col in df.columns])
    create_table_query = f'CREATE TABLE IF NOT EXISTS `{table_name}` ({columns})'
    cursor.execute(create_table_query)

    # Insert DataFrame data into the MySQL table
    for _, row in df.iterrows():
        # Convert row to tuple and handle NaN/None explicitly
        values = tuple(None if pd.isna(x) else x for x in row)
        sql = f"INSERT INTO `{table_name}` ({', '.join(['`' + col + '`' for col in df.columns])}) VALUES ({', '.join(['%s'] * len(row))})"
        cursor.execute(sql, values)

    # Commit the transaction for the current CSV file
    conn.commit()

# Close the connection
conn.close()

Processing customers.csv
NaN values before replacement:
customer_id                 0
customer_unique_id          0
customer_zip_code_prefix    0
customer_city               0
customer_state              0
dtype: int64

Processing orders.csv
NaN values before replacement:
order_id                            0
customer_id                         0
order_status                        0
order_purchase_timestamp            0
order_approved_at                 160
order_delivered_carrier_date     1783
order_delivered_customer_date    2965
order_estimated_delivery_date       0
dtype: int64

Processing sellers.csv
NaN values before replacement:
seller_id                 0
seller_zip_code_prefix    0
seller_city               0
seller_state              0
dtype: int64

Processing products.csv
NaN values before replacement:
product_id                      0
product category              610
product_name_length           610
product_description_length    610
product_photos_qty            610
prod

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import mysql.connector 

db_conn = mysql.connector.connect(host = "127.0.0.1",
                                 username = "root",
                                 password = '@UmaHustler019',
                                 database = "ecommerce_target")

cur = db_conn.cursor()



# 1. List all unique cities where customers are located.

In [3]:
query1 = """SELECT distinct customer_city FROM customers"""

cur.execute(query1)

data = cur.fetchall()

#print(data)

df = pd.DataFrame(data)
df.head()


,0
0,franca
1,sao bernardo do campo
2,sao paulo
3,mogi das cruzes
4,campinas


# 2.Count the number of orders placed in 2017

In [8]:
query2 = """select COUNT(order_id) from orders
where year(order_purchase_timestamp) = 2017"""

cur.execute(query2)

data = cur.fetchall()

print(data[0][0])  #calling only value not tuple on index 0


135303


# 3. Find the total sales per category.

In [24]:
query3 = """SELECT 
    UPPER(products.product_category) AS category, 
    ROUND(SUM(payments.payment_value), 2) AS sales
FROM 
    products 
JOIN 
    order_items 
ON 
    products.product_id = order_items.product_id
JOIN 
    payments 
ON 
    payments.order_id = order_items.order_id
GROUP BY 
    category;

"""

cur.execute(query3)

data = cur.fetchall()

#print(data)  #calling only value not tuple on index 0

df = pd.DataFrame(data,columns =["category","sales"])
df.tail(5)


,category,sales
69,CDS MUSIC DVDS,32384.61
70,LA CUISINE,78665.31
71,FASHION CHILDREN'S CLOTHING,21213.09
72,PC GAMER,58709.61
73,INSURANCE AND SERVICES,8761.77


In [27]:
db_conn.dropisnull() # python query will not run on this database,good to learn n experinece

AttributeError: 'CMySQLConnection' object has no attribute 'dropisnull'

# 4.  Calculate the percentage of orders that were paid in installments.

In [10]:
query4 = """SELECT (SUM(CASE WHEN payment_installments >=1 then 1 else 0 end))/COUNT(*)
* 100 FROM payments"""

cur.execute(query4)

data = cur.fetchall()

print(data)


[(Decimal('99.9981'),)]


# 5. Count the number of customers from each state. 

In [11]:
query5 = """SELECT count(customer_id) as count_of_customers , customer_state  FROM customers
group by customer_state ;"""

cur.execute(query5)

data = cur.fetchall()

#print(data)


df = pd.DataFrame(data, columns =["No.of Customers","State"])
#df = df.sort_values(by = "No.of Customers",ascending = False)
df.head()



,No.of Customers,State
0,125238,SP
1,10911,SC
2,34905,MG
3,15135,PR
4,38556,RJ


# 6. Calculate the number of orders per month in 2018.

In [4]:
query6 = """select monthname(order_purchase_timestamp) as months , COUNT(order_id) as no_of_orders from ecommerce_target.orders
where year(order_purchase_timestamp) = 2018
GROUP BY months ;
"""

cur.execute(query6)

data = cur.fetchall()

df = pd.DataFrame(data, columns =["Months","Orders_Count"])
df



,Months,Orders_Count
0,July,18876
1,August,19536
2,February,20184
3,June,18501
4,March,21633
5,January,21807
6,May,20619
7,April,20817
8,September,48
9,October,12


# Find the average number of products per order, grouped by customer city.


In [25]:
query = """with count_per_order as 
(select orders.order_id, orders.customer_id, count(order_items.order_id) as oc
from orders join order_items
on orders.order_id = order_items.order_id
group by orders.order_id, orders.customer_id)

select customers.customer_city, round(avg(count_per_order.oc),2) average_orders
from customers join count_per_order
on customers.customer_id = count_per_order.customer_id
group by customers.customer_city order by average_orders desc
"""

cur.execute(query)

data = cur.fetchall()
df = pd.DataFrame(data,columns = ["customer city", "average products"])
df.head(10)

,customer city,average products
0,padre carvalho,63.00
1,celso ramos,58.50
2,datas,54.00
3,candido godoi,54.00
4,matias olimpio,45.00
5,cidelandia,36.00
6,curralinho,36.00
7,picarra,36.00
8,morro de sao paulo,36.00
9,teixeira soares,36.00


# 7.Calculate the percentage of total revenue contributed by each product category.

In [5]:
query7 = """
SELECT UPPER(p.product_category) AS category, 
       ROUND((SUM(pay.payment_value) / (SELECT SUM(payment_value) FROM payments)) * 100, 2) AS revenue_percentage
FROM products AS p
JOIN order_items AS o ON o.product_id = p.product_id
JOIN payments AS pay ON pay.order_id = o.order_id
GROUP BY category 
ORDER BY revenue_percentage DESC;
"""

cur.execute(query7)
data = cur.fetchall()
df = pd.DataFrame(data, columns=["Category", "percentage distribution"])
df


,Category,percentage distribution
0,BED TABLE BATH,96.28
1,HEALTH BEAUTY,93.18
2,COMPUTER ACCESSORIES,89.13
3,FURNITURE DECORATION,80.40
4,WATCHES PRESENT,80.35
...,...,...
69,PC GAMER,0.12
70,HOUSE COMFORT 2,0.10
71,CDS MUSIC DVDS,0.07
72,FASHION CHILDREN'S CLOTHING,0.04


# 8. Identify the correlation between product price and the number of times a product has been purchased.

In [8]:
query8 = """

select p.product_category as category , count(o.product_id) as count , round(avg(o.price),2) as each_product_avg_price FROM products as p
JOIN order_items as o
on o.product_id = p.product_id 
group by category ;

"""

cur.execute(query8)
data = cur.fetchall()
df = pd.DataFrame(data, columns=["Category", "order_count", "price"])
arr1 = df["order_count"]
arr2 = df["price"]
a = np.corrcoef([arr1, arr2])
print("the correlation is", a[0][-1])
print()
print(a)

the correlation is -0.10631514167157562

[[ 1.         -0.10631514]
 [-0.10631514  1.        ]]


# 9.Calculate the total revenue generated by each seller, and rank them by revenue.

In [19]:

query9 = """
SELECT 
    order_items.seller_id, 
    ROUND(SUM(payments.payment_value), 2) AS revenue, 
    DENSE_RANK() OVER (ORDER BY ROUND(SUM(payments.payment_value), 2) DESC) AS ranks
FROM 
    order_items 
JOIN 
    payments 
ON 
    order_items.order_id = payments.order_id 
GROUP BY 
    order_items.seller_id;

"""

cur.execute(query9)
data = cur.fetchall()
df = pd.DataFrame(data,columns=["seller_id","revenue","rank"])
df.head(5)


,seller_id,revenue,rank
0,7c67e1448b00f6e969d365cea6b010ab,4564502.17,1
1,1025f0e2d44d7041d6cf58b6550e0bfa,2773998.36,2
2,4a3ca9315b744ce9f8e9374361493884,2711207.43,3
3,1f50f920176fa81dab994f9023523100,2612280.78,4
4,53243585a1d6dc2643021fd1853d8905,2564127.72,5


# 10.Calculate the moving average of order values for each customer over their order history.

In [4]:
query9 = """
SELECT customer_id, order_purchase_timestamp, payment,
       AVG(payment) OVER (PARTITION BY customer_id ORDER BY order_purchase_timestamp ROWS BETWEEN 2 PRECEDING AND CURRENT ROW) AS avg_payment
FROM (
    SELECT o.order_purchase_timestamp, o.customer_id, p.payment_value AS payment
    FROM payments AS p
    JOIN orders AS o ON o.order_id = p.order_id
) AS a;


"""

cur.execute(query9)
data = cur.fetchall()
df = pd.DataFrame(data,columns=["coutomer_id","order_purchased_time","payment","Avg_payment"])
df


,coutomer_id,order_purchased_time,payment,Avg_payment
0,00012a2ce6f8dcda20d059ce98491703,2017-11-14 16:08:26,114.74,114.739998
1,00012a2ce6f8dcda20d059ce98491703,2017-11-14 16:08:26,114.74,114.739998
2,00012a2ce6f8dcda20d059ce98491703,2017-11-14 16:08:26,114.74,114.739998
3,00012a2ce6f8dcda20d059ce98491703,2017-11-14 16:08:26,114.74,114.739998
4,00012a2ce6f8dcda20d059ce98491703,2017-11-14 16:08:26,114.74,114.739998
...,...,...,...,...
934969,ffffe8b65bbe3087b653a978c870db99,2017-09-29 14:07:03,18.37,18.370001
934970,ffffe8b65bbe3087b653a978c870db99,2017-09-29 14:07:03,18.37,18.370001
934971,ffffe8b65bbe3087b653a978c870db99,2017-09-29 14:07:03,18.37,18.370001
934972,ffffe8b65bbe3087b653a978c870db99,2017-09-29 14:07:03,18.37,18.370001


# 11.Calculate the cumulative sales per month for each year.

In [6]:
query10="""SELECT 
    years, 
    months, 
    sales, 
    SUM(sales) OVER (ORDER BY years, months) AS cumulative_sales
FROM (
    SELECT 
        YEAR(o.order_purchase_timestamp) AS years,
        MONTH(o.order_purchase_timestamp) AS months,
        ROUND(SUM(p.payment_value), 2) AS sales 
    FROM 
        orders AS o 
    JOIN 
        payments AS p ON o.order_id = p.order_id
    GROUP BY 
        years, months
    ORDER BY 
        years, months
) AS a; """

cur.execute(query9)
data = cur.fetchall()
df = pd.DataFrame(data,columns=["year","month","sales","cummulative_sales"])
df

     

,year,month,sales,cummulative_sales
0,00012a2ce6f8dcda20d059ce98491703,2017-11-14 16:08:26,114.74,114.739998
1,00012a2ce6f8dcda20d059ce98491703,2017-11-14 16:08:26,114.74,114.739998
2,00012a2ce6f8dcda20d059ce98491703,2017-11-14 16:08:26,114.74,114.739998
3,00012a2ce6f8dcda20d059ce98491703,2017-11-14 16:08:26,114.74,114.739998
4,00012a2ce6f8dcda20d059ce98491703,2017-11-14 16:08:26,114.74,114.739998
...,...,...,...,...
934969,ffffe8b65bbe3087b653a978c870db99,2017-09-29 14:07:03,18.37,18.370001
934970,ffffe8b65bbe3087b653a978c870db99,2017-09-29 14:07:03,18.37,18.370001
934971,ffffe8b65bbe3087b653a978c870db99,2017-09-29 14:07:03,18.37,18.370001
934972,ffffe8b65bbe3087b653a978c870db99,2017-09-29 14:07:03,18.37,18.370001


# 12.Calculate the year-over-year growth rate of total sales.

In [10]:
query11 = """
SELECT 
    years,
    sales,
    (sales - LAG(sales, 1) OVER (ORDER BY years)) / LAG(sales, 1) OVER (ORDER BY years) * 100 AS yoy_growth
FROM (
    SELECT 
        YEAR(o.order_purchase_timestamp) AS years,
        SUM(p.payment_value) AS sales
    FROM 
        payments AS p 
    JOIN 
        orders AS o ON o.order_id = p.order_id
    GROUP BY 
        YEAR(o.order_purchase_timestamp)
) AS yearly_sales;
"""

cur.execute(query11)
data = cur.fetchall()
df = pd.DataFrame(data)
df


,0,1,2
0,2016,5.342611e+05,NaN
1,2017,6.524772e+07,12112.703760
2,2018,7.829787e+07,20.000924


# 12.Identify the top 3 customers who spent the most money in each year.

In [23]:
query12 = """
SELECT 
    customer_id, 
    year, 
    total_spent
FROM (
    SELECT 
        customer_id, 
        year, 
        total_spent,
        DENSE_RANK() OVER (PARTITION BY year ORDER BY total_spent DESC) AS customer_rank
    FROM (
        SELECT 
            o.customer_id, 
            YEAR(o.order_purchase_timestamp) AS year, 
            SUM(p.payment_value) AS total_spent
        FROM 
            orders o
        JOIN 
            payments p ON o.order_id = p.order_id
        GROUP BY 
            o.customer_id, YEAR(o.order_purchase_timestamp)
    ) AS yearly_spending
) AS ranked_customers
WHERE customer_rank <= 3;
"""

cur.execute(query12)
data = cur.fetchall()
df = pd.DataFrame(data, columns = ["id","year","payment"])
#df = pd.DataFrame(data)
df


,id,year,payment
0,a9dc96b027d1252bbac0a9b72d837fc6,2016,12811.950439
1,1d34ed25963d5aae4cf3d7f3a4cda173,2016,12606.659912
2,4a06381959b6670756de02e07b83815f,2016,11050.020264
3,1617b1357756262bfa56ab541c47bc16,2017,122976.720703
4,c6e2731c5b391845f6800c97401a43a9,2017,62363.790527
5,3fd6777bbce08a352fddd04e4a7cc8f6,2017,60539.941406
6,ec5b2ba62e574342386871631fafd3fc,2018,65473.918945
7,f48d464a0baaea338cb25f816991ab1f,2018,62299.889648
8,e0a2412720e9ea4f26c1ac985f6a7358,2018,43284.959473
